# Data Component Use Case for Permafrost and Hillslope Processes

## Introduction

Permafrost is defined as any material (rock or soil) that remains below 0°C for two or more consecutive years. This Jupyter notebook demonstrates how to use several [CSDMS data components](https://csdms.colorado.edu/wiki/DataComponents) to download topography, snow, and temperature data to calculate the permafrost active layer thickness and simulate the hillslope evolution process for a study area in Alaska. 

In this notebook, it includes the following sections:
- [Initial Setup](#setup)
  
  This section will help install several Python packages and create the input/output folders.
  <br>
- [Step 1: Download Datasets](#step1) 

  This section will download the topography, temperature and snow datasets.
  <br>
- [Step 2: Calculate Activate Layer Thickness ](#step2)

  This section will use temperature and snow datasets to calculate the activate layer thickness of the study area.  
  <br>
- [Step 3: Simulate Hillslope Diffusion](#step3)

  This section will use the activate layer thickness and topography data as the input to simulate the hillslope diffusion process.
  <br>


**Suggested Citation**: Gan, T., Tucker, G. E., Overeem, I., Pierce, E. (2023). Data Component Use Case for Permafrost and Hillslope Processes, HydroShare, https://www.hydroshare.org/resource/c4ebe3515a894446bf26aba0aedd7fdd/

<a id='setup'></a>
## Initial Setup

We will need to install several python packages and create folders for this notebook. After you install the packages, you will need to run a helper function to create API Key files for the Topography and ERA5 data components.

### Install Packages (on Local PC)
If you want to run this notebook on your PC, you can run the following command which will create a separate conda environment named "permafrost_usecase" and install all the required packages for you. After the installation, please make sure to activate the environment to run this notebook.

In [ ]:
# ! conda env create --file=environment.yml

### Install API key files
For the ERA5 and Topography data components, there is a need to create API key files to download the datasets. The install_api_key( ) function will ask for your [CDS API Key](https://cds.climate.copernicus.eu/api-how-to) and [Open Topography API Key](https://opentopography.org/blog/introducing-api-keys-access-opentopography-global-datasets) to create API key files. Please make sure you have already obtained those API Keys before you run this helper function. 


In [ ]:
from utils import install_api_key
install_api_key()

### Create folders
We will first import all the python packages and then create three folders for this notebook:
- **configuration file folder**: this folder includes several configuration files which will be used by the data components. In this example, we have prepared these configuration files ('dem_config.yaml' and 'era5_config.yaml') and put them in this folder. 
- **cache folder**: this folder stores the downloaded data files.
- **results folder**: this folder stores the final results.

In [ ]:
# import packages 
import os
import warnings

import numpy as np
import pandas as pd
import xarray 
import cftime
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import LightSource
import imageio.v2 as imageio
from IPython.display import Video

from pymt.models import Topography, Era5, Ku
from landlab import RasterModelGrid, imshow_grid
from landlab.components import DepthDependentDiffuser

warnings.simplefilter(action='ignore', category=FutureWarning)
plt.rcParams.update({'font.size': 12})

In [ ]:
# create folders
study_area = 'alaska'

config_dir = os.path.join(os.getcwd(), 'config_files_{}'.format(study_area))
results_dir = os.path.join(os.getcwd(), 'results_{}'.format(study_area)) 
cache_dir = os.path.join(os.getcwd(),'cache_{}'.format(study_area))


for folder in [config_dir, results_dir, cache_dir]:
    if not os.path.isdir(folder):
        os.mkdir(folder)
        print(folder)

<a id='step1'></a>
## Step 1 Download Datasets

### Background 

Permafrost covers nearly 85% of Alaska. The map below shows the permafrost distribution in this area (map source from [Jorgenson et al.](https://permafrost.gi.alaska.edu/sites/default/files/AlaskaPermafrostMap_Front_Dec2008_Jorgenson_etal_2008.pdf)). A warming climate brought higher temperatures which may cause some permafrost to thaw. This can lead to geologic hazards such as landslides, ground subsidence, erosion and other severe surface distortions.

In this use case, we will run a model to calculate the active layer thickness for a study area in Alaska. Active layer is the top layer of the permafrost where the annual maximum temperature reaches 0°C and the temperature shift over diurnal and seasonal cycles. Then we will use the active layer thickness as the input to simulate the hillslope diffusion process for the study area. And we need to prepare the following datasets for this use case.
- OpenTopography DEM 
- ERA5 temperature 
- ERA5 snow 


<img src="https://github.com/gantian127/permafrost_usecase/blob/master/permafrost.png?raw=true" width="700">

### OpenTopography DEM 

We will use the [Topography data component](https://csdms.colorado.edu/wiki/Model:Topography_Data_Component) to download the Digital Elevation Model (DEM) data with 30m resolution. The 'dem_config.yaml' file includes the parameter settings of this data component. The following cells demonstrate how to use the configuration file to initialize a data component and how to use the variable and grid related methods of this data component to get the metadata as well as the data values. 

In [ ]:
# initialize Topography data component
dem = Topography()
dem.initialize(os.path.join(config_dir, 'dem_config.yaml'))

In [ ]:
# get DEM variable info
var_name = dem.output_var_names[0]
var_unit = dem.var_units(var_name)
var_location = dem.var_location(var_name)
var_type = dem.var_type(var_name)
var_grid = dem.var_grid(var_name)
var_itemsize = dem.var_itemsize(var_name)
var_nbytes = dem.var_nbytes(var_name)
print('variable_name: {} \nvar_unit: {} \nvar_location: {} \nvar_type: {} \nvar_grid: {} \nvar_itemsize: {}' 
            '\nvar_nbytes: {} \n'. format(var_name, var_unit, var_location, var_type, var_grid, var_itemsize, var_nbytes))

In [ ]:
# get DEM grid info 
dem_grid_ndim = dem.grid_ndim(var_grid) 
dem_grid_type = dem.grid_type(var_grid)
dem_grid_shape = dem.grid_shape(var_grid)
dem_grid_spacing = dem.grid_spacing(var_grid)
dem_grid_origin = dem.grid_origin(var_grid)

print('grid_ndim: {} \ngrid_type: {} \ngrid_shape: {} \ngrid_spacing: {} \ngrid_origin: {}'.format(
    dem_grid_ndim, dem_grid_type, dem_grid_shape, dem_grid_spacing, dem_grid_origin))

In [ ]:
# get DEM variable data
dem_data = dem.get_value(var_name)
dem_data_2D = dem_data.reshape(dem_grid_shape)

# get X, Y extent for plot
min_y, min_x = dem_grid_origin
max_y = min_y + dem_grid_spacing[0]*(dem_grid_shape[0]-1)
max_x = min_x + dem_grid_spacing[1]*(dem_grid_shape[1]-1)
dy = dem_grid_spacing[0]/2
dx = dem_grid_spacing[1]/2
dem_extent = [min_x - dx, max_x + dx, min_y - dy, max_y + dy]

# plot DEM data as hillshade plot
fig, ax = plt.subplots(figsize=(8,9))
ls = LightSource(azdeg = 90, altdeg = 30)
shaded_dem = ls.hillshade(np.array(dem_data_2D))

ax.imshow(shaded_dem, cmap = 'Greys',extent=dem_extent)
ax.ticklabel_format(useOffset=False)
ax.set_xlabel('latitude [degree_north]')
ax.set_ylabel('longitude [degree_east]')
ax.title.set_text('Study area of Eightmile Lake')
plt.show()

### ERA5 Snow and Temperature
We will use the [ERA5 data component](https://csdms.colorado.edu/wiki/Model:ERA5_Data_Component) to download the monthly mean snow water equivalent, snow density and 2m air temperature datasets for the study area with 0.25 degrees (27-28km) resolution. We will download two datasets for 1980-1989 and 2010-2019 separately.  



'era5_1980_config.yaml' and 'era5_2010_config.yaml' files include the parameter settings for two ERA5 data components. The following cells demonstrate how to use the configuration file to initialize an ERA5 data component and how to use the variable, grid and time related methods to get the metadata as well as the data values. 

You'll notice that although the ERA5 and Topography data components download the datasets from different sources, they are using the same methods to get information from the datasets. Please note that sometimes the request for ERA5 data may be queued which may take a while (>10min) to get the data downloaded.

In [ ]:
# initialize ERA5 data components 

# 1980-1989 data
era5 = Era5()
era5.initialize(os.path.join(config_dir,'era5_1980_config.yaml'))

# 2010-2019 data
era5_2 = Era5()
era5_2.initialize(os.path.join(config_dir,'era5_2010_config.yaml'))

In [ ]:
# get ERA5 variable info
for var_name in era5.output_var_names:
    var_unit = era5.var_units(var_name)
    var_location = era5.var_location(var_name)
    var_type = era5.var_type(var_name)
    var_grid = era5.var_grid(var_name)
    var_itemsize = era5.var_itemsize(var_name)
    var_nbytes = era5.var_nbytes(var_name)
    print('variable_name: {} \nvar_unit: {} \nvar_location: {} \nvar_type: {} \nvar_grid: {} \nvar_itemsize: {}' 
            '\nvar_nbytes: {} \n'. format(var_name, var_unit, var_location, var_type, var_grid, var_itemsize, var_nbytes))

In [ ]:
# get ERA5 grid info
era5_grid_ndim = era5.grid_ndim(var_grid) 
era5_grid_type = era5.grid_type(var_grid)
era5_grid_shape = era5.grid_shape(var_grid)
era5_grid_spacing = era5.grid_spacing(var_grid)
era5_grid_origin = era5.grid_origin(var_grid)

print('grid_ndim: {} \ngrid_type: {} \ngrid_shape: {} \ngrid_spacing: {} \ngrid_origin: {}'.format(
    era5_grid_ndim, era5_grid_type, era5_grid_shape, era5_grid_spacing, era5_grid_origin))

In [ ]:
# get ERA5 time info
era5_start_time = era5.start_time
era5_end_time = era5.end_time
era5_time_unit = era5.time_units

print('start_time:{} \nend_time:{} \ntime_unit:{}'.format(
    era5_start_time, era5_end_time, era5_time_unit))

In [ ]:
# get ERA5 variables data and plot (at the first time step)
fig = plt.figure(figsize=(14,12)) 
nrows, ncols = 2, 2
i = 1

for var_name in era5.output_var_names:
    ax = fig.add_subplot(nrows, ncols, i)
    var_unit = era5.var_units(var_name)
    
    # get variable data    
    era5_data = era5.get_value(var_name)
    era5_data_2D = era5_data.reshape(era5_grid_shape)
    
    # get X, Y extent for plot
    min_y, min_x = era5_grid_origin
    max_y = min_y + era5_grid_spacing[0]*(era5_grid_shape[0]-1)
    max_x = min_x + era5_grid_spacing[1]*(era5_grid_shape[1]-1)
    dy = era5_grid_spacing[0]/2
    dx = era5_grid_spacing[1]/2
    era5_extent = [min_x - dx, max_x + dx, min_y - dy, max_y + dy]

    # plot data
    im = ax.imshow(era5_data_2D, extent=era5_extent, cmap='Blues')
    ax.title.set_text('{} ({})'.format(var_name,var_unit ))
    ax.set_xlabel('longitude [degree_east]')
    ax.set_ylabel('latitude [degree_north]')
    cbar = plt.colorbar(im, ax=ax)
    
    i += 1

<a id='step2'></a>
## Step 2 Calculate Activate Layer Thickness

In this section, we will use the ERA5 datasets to prepare inputs and run Ku model (Kudryavtsev et al.,1974) to calculate the active layer thickness. The Ku model provides a steady-state solution under the assumption of sinusoidal air temperature forcing. It considers snow, vegetation, and soil layers as thermal damping to variation of air temperature. The layer of soil is considered to be a homogeneous column with different thermal properties in the frozen and thawed states. One of its main outputs is the annual active layer thickness at the top of permafrost.

Given the fact that the air temperature has an increasing trend over the years, we want to explore whether this change has an impact on the active layer thickness for the study area. So we will prepare two sets of inputs for 1980-1989 and 2010-2019 separately and compare the model results.

This section includes the following tasks:
- Get ERA5 data for study area
- Prepare temperature and snow inputs 
- Run Ku model
- Results analysis



### Get ERA5 data for study area

From the Topography and ERA5 data plots, it can be found that the study area falls in the upper left grid of the ERA5 data. So we will get the time series of air temperature, snow density and snow depth from the two ERA5 data components for the study area. 

In [ ]:
# create dataframe to store time series data
era5_df = pd.DataFrame(columns = ['temp','swe','dens','time'])
time_steps = 12*10  # 10 years of monthly data 

for data_comp in [era5, era5_2]:
    
    for i in range(0, time_steps):  
        # get values
        temp = data_comp.get_value('2 metre temperature')
        swe = data_comp.get_value('Snow depth')
        dens = data_comp.get_value('Snow density') 
        time = cftime.num2pydate(data_comp.time, data_comp.time_units)

        # add new row to dataframe
        era5_df.loc[len(era5_df)]=[temp[0], swe[0], dens[0], time]

        # update to next time step
        data_comp.update()
    
era5_df = era5_df.set_index('time') 

In [ ]:
# plot time series data
for start, end in [(1980,1989),(2010,2019)]:
    ax = era5_df[(start<=era5_df.index.year) & (era5_df.index.year<=end)].plot(
        y=['temp','swe','dens'], subplots=True, figsize=(8,7), xlabel='',
        legend=None, title = 'ERA5 monthly mean data for {}-{}'.format(start,end),)
    ax[0].set_title('air temperature (K)')
    ax[1].set_title('snow water equivalent (m)') 
    ax[2].set_title('snow density (kg/m3)')

### Prepare temperature and snow inputs

Now, we will use the time series data for the study area to prepare inputs for Ku model, which includes annual mean temperature, temperature amplitude and snow thickness. 

In [ ]:
# create a dataframe to store input data
input_df = pd.DataFrame(columns = ['temp_mean','temp_amp', 'snow_h'])

# calculate annual mean temperature and temperature amplitude
input_df['temp_mean'] = era5_df['temp'].groupby(pd.Grouper(freq='Y')).mean() - 273.15
input_df['temp_amp'] = era5_df['temp'].groupby(pd.Grouper(freq='Y')).max() - era5_df['temp'].groupby(pd.Grouper(freq='Y')).mean()

# calculate snow thickness (SWE * water_dens/snow_dens)
water_dens = 1000 # kg/m3
snow_df = era5_df['swe'] * water_dens / era5_df['dens']
snow_df = snow_df[(snow_df.index.month<=5) | (snow_df.index.month>=9)]  # remove summer month
input_df['snow_h'] = snow_df.groupby(pd.Grouper(freq='Y')).mean()

input_df=input_df.round(3).dropna()

# plot the input datasets
for start, end in [(1980,1989),(2010,2019)]:
    ax = input_df[(start<=input_df.index.year) & (input_df.index.year<=end)].plot(
        y=['temp_mean','temp_amp','snow_h'], subplots=True, figsize=(10,8), xlabel='',legend=None, 
        title = 'Temperature and snow input for {}-{}'.format(start,end))
    ax[0].set_title('annual mean temperature (°C)')
    ax[1].set_title('temperature amplitude (°C)') 
    ax[2].set_title('snow thickness (m)')



Let's look at these plots above. What do you find from them? It can be seen that the annual mean temperature tends to increase while the temperature amplitude and snow thickness decreased during 2010-2019. How will this change impact the active layer thickness? Will it become thicker because of temperature increase? Let's run Ku model to find it out.

### Run Ku model

Ku model has been implemented as one of the [pymt model components](https://pymt.readthedocs.io/en/latest/models.html#model-components). And we will import Ku model component from pymt for model run.

Since the model or data is wrapped with the [Basic Model Interface](https://bmi.readthedocs.io/en/latest/) to become a component under the [pymt](https://pymt.readthedocs.io/en/latest/index.html) modeling framework, the way to control or query a model or a data component is the same. For example, the methods to initialize, update and get value for Ku model component are the same as the ERA5 data component. 

In [ ]:
# create a dataframe to store results
active_layer = pd.DataFrame(columns = ['active_h'], index = input_df.index)

# run Ku model
for start, end in [(1980,1989), (2010,2019)]:   
    print('Simulation for {}-{}'.format(start,end))
    
    # get input data  
    input_data = input_df[(input_df.index.year>=start) & (input_df.index.year<=end)]    
    
    # setup model
    ku = Ku()  
    args = ku.setup(start_year=start, end_year=end, lat=63.88, lon=-149.25)
    ku.initialize(*args)
    
    # run model
    for index, row in input_data.iterrows():        
        ku.set_value("atmosphere_bottom_air__temperature", row['temp_mean'])
        ku.set_value("atmosphere_bottom_air__temperature_amplitude", row['temp_amp'])
        ku.set_value("snowpack__depth", row['snow_h'])
        ku.update()
        
        # store result
        active_layer.loc[index] = ku.get_value('soil__active_layer_thickness')[0]

print('Simulation is done!')

In [ ]:
# plot Ku model result
for start, end in [(1980,1989),(2010,2019)]:
    ax = active_layer[(start<=active_layer.index.year) & (active_layer.index.year<=end)].plot.bar(
        y=['active_h'], subplots=True, figsize=(7,5), xlabel='',legend=None)
    ax[0].set_ylabel('Thickness (m)')
    ax[0].set_xticklabels(np.arange(start, end+1, step=1),rotation=0)
    ax[0].set_title('Active Layer Thickness for {}-{}'.format(start,end))
    

### Results analysis

In the active layer thickness plot, the change of temperature and snow conditions during 2010-2019 doesn't impact much on the active layer thickness to make it increase as we expected. 

Why is that? To find out the reason, we will first calculate the 10-year average of the input and output data for 1980-1989 and 2010-2019 and do some model experiments. 

In [ ]:
# get 10 year average of input and output for 1980-1989 and 2010-2019
ave_df = pd.DataFrame(columns=['temp_mean','temp_amp','snow_h','active_h'], index=[1980,2010])

for start, end in [(1980,1989),(2010,2019)]:
    ave_input = input_df[(input_df.index.year>=start) & (input_df.index.year<=end)].mean().round(3)
    ave_output = active_layer[(start<=active_layer.index.year) & (active_layer.index.year<=end)].mean().round(3)
    ave_df.loc[start] = [ave_input['temp_mean'], ave_input['temp_amp'],
                         ave_input['snow_h'], ave_output['active_h']]
ave_df

The 10-year average data proves what we found from the temperature and snow inputs plot and the active layer thickness plot. The increased annual mean temperature and decreased temperature amplitude as well as snow thickness didn't lead to much change of the active layer thickness (about 4mm difference). 

We will run several model experiments to find out the impact of those model inputs on the active layer thickness. We will run the model using 10-year average input for 1980-1989 and 2010-2019. The model run for 1980-1989 will be taken as the "base" experiment for comparison. Based on the model inputs of 1980-1989, we will change its annual mean temperature, temperature amplitude and snow thickness value separately by replacing them with the 10-year average input of 2010-2019.

In [ ]:
# experiment inputs
experiment = {
    '1980-1989':[-3.849, 16.99, 0.355],
    '2010-2019':[-2.563, 16.234, 0.247],
    'change of annual mean temperature': [-2.563, 16.99, 0.355],
    'change of temperature amplitude':[-3.849, 16.234, 0.355],
    'change of snow thickness': [-3.849, 16.99, 0.247],
}

In [ ]:
# run experiments
for key, value in experiment.items():    
    ku = Ku()
    args = ku.setup(T_air=value[0], A_air=value[1], h_snow=value[2], lat=63.88, lon=-149.25)
    ku.initialize(*args)
    ku.update()

    print('Result for {}: {}m'.format(key, ku.get_value('soil__active_layer_thickness')[0].round(3)))

The experiment results showed that it can lead to an increase of the active layer thickness by only increasing the annual temperature. But especially if the snow thickness decreases, its insulating capacity in mid and late winter will decrease, and as such the active layer thickness will also decrease. Therefore, the respective change of warming temperature versus a decreasing snow thickness can act in opposing direction and thereby minimize changes for the active layer thickness. This phenomenon was also observed with field datasets and studied by several researchers at other study sites (Garnello et al., 2021; Zhang, 2005).


<a id='step3'></a>
## Step 3  Simulate Hillslope Diffusion

In this section, we will use the topography DEM and the active layer thickness as the inputs for a hillslope diffusion model. The [Landlab](https://landlab.readthedocs.io/en/master/) component [DepthDependentDiffuser]() will be used for simulation. This component implements a depth and slope dependent linear diffusion rule in the style of Johnstone and Hilley (2015). 

This section includes the following tasks:
- Create model grid and data fields
- Initialize and run model component 
- Results visualization

### Create model grid and data fields

In [ ]:
# create a model grid
model_grid = RasterModelGrid(dem_grid_shape,xy_spacing=30)

# add soil depth field and set it as active layer thickness
soil_depth = model_grid.add_zeros("soil__depth", at="node", clobber=True)

active_h = 0.992  # result for 2010-2019 with 10-year average input
model_grid.at_node["soil__depth"].fill(active_h)

# add soil production rate field and set it as 0
production = model_grid.add_zeros('soil_production__rate', at='node', clobber=True)

# add elevation field using topography DEM data
elevation = model_grid.add_field('node', 'topographic__elevation', dem_data.astype('float'), units = 'm', clobber=True)

# plot elevation data field
fig = plt.figure(figsize=(8,6))
ax = plt.gca()
ax.invert_yaxis()
imshow_grid(model_grid,
            'topographic__elevation',
            plot_name='Elevation of Eight Mile Lake',
            var_name='elevation (m)',
            )
ax.set_xlabel('east-west distance (m)')
ax.set_ylabel('north-south distance (m)')
fig.savefig(os.path.join(results_dir, 'elev_0.png'.format(time)))

### Initialize and run model component 

In [ ]:
# set parameters 
linear_diffusivity = 1e-2  # m^2/yr
transport_decay_depth = 1  # m
dt = 1  # time step (year)
nt = 1000  # total steps (years) 

# get original dem data
ref_data = np.copy(elevation)

# initialize the landlab component
ddiff = DepthDependentDiffuser(model_grid, linear_diffusivity, transport_decay_depth)

for time in range(1,nt+1):   
    # run model
    ddiff.run_one_step(dt)
    
    # save result
    if time % 50 == 0:
        print('Completed for {} years'.format(time))
        im_data = model_grid.at_node['topographic__elevation'] - ref_data

        fig = plt.figure(figsize=(8,6))
        ax = plt.gca()
        ax.invert_yaxis()
        imshow_grid(model_grid,
            im_data,
            plot_name='Simulated elevation change after {} years'.format(time),
            var_name='elevation (m)',
            vmin=-0.1,
            vmax=0.1,
            cmap = 'winter_r',
        )      
        ax.set_xlabel('east-west distance (m)')
        ax.set_ylabel('north-south distance (m)')
        plt.close(fig)
        fig.savefig(os.path.join(results_dir, 'elev_{}.png'.format(time)))
        
            
print('Simulation is done!')

### Results visualization

Let's run the cells bellow to visualize the results. The plot shows the simulated elevation after 1000 years of hillslope diffusion process. The video shows the elevation change during the simulation period. You will find the vertical stripe pattern from the video which is mainly caused by the similar pattern from the terrain elevation data and can be improved with other datasets in a higher resolution (e.g., Lidar DEM dataset). 

In [ ]:
# plot result
fig = plt.figure(figsize=(8,6))
ax = plt.gca()
ax.invert_yaxis()
imshow_grid(model_grid,
            'topographic__elevation',
            plot_name='Simulated Elevation after 1000 years',
            var_name='elevation (m)',
            )
ax.set_xlabel('east-west distance (m)')
ax.set_ylabel('north-south distance (m)')

In [ ]:
# Make a short video
img_files = [os.path.join(results_dir, file) for file in os.listdir(results_dir) if '.png' in file]
img_files.sort(key=lambda x: os.path.getmtime(x))

with imageio.get_writer(os.path.join(results_dir,'hillslope.mp4'), mode='I', fps=1,macro_block_size=None ) as writer:
    for f in img_files:
        im = imageio.imread(os.path.join(results_dir, f))
        writer.append_data(im)

writer.close()

# Display the video
Video("./results_alaska/hillslope.mp4", embed=True, width=500, height=500)

## References
- Anisimov, O. A., Shiklomanov, N. I., & Nelson, F. E. (1997). Global warming and active-layer thickness: results from transient general circulation models. Global and Planetary Change, 15(3-4), 61-77. https://doi.org/10.1016/S0921-8181(97)00009-X
- Garnello, A., Marchenko, S., Nicolsky, D., Romanovsky, V., Ledman, J., Celis, G., Schädel, C., Luo, Y., & Schuur, E. A. G. (2021). Projecting Permafrost Thaw of Sub-Arctic Tundra With a Thermodynamic Model Calibrated to Site Measurements. Journal of Geophysical Research: Biogeosciences, 126(6), e2020JG006218. https://doi.org/https://doi.org/10.1029/2020JG006218![image.png](attachment:image.png)
- Johnstone, S., Hilley, G. (2015). Lithologic control on the form of soil-mantled hillslopes, Geology 43(1), 83-86. https://doi.org/10.1130/G36052.1
- Kudryavtsev, V. A., Garagulya, L. S., Kondrat'yeva, K. A., Melamed, V. G. (1974). 
Fundamentals of frost forecasting in geological engineering investigations
Nauka, Moscow, p. 431. https://apps.dtic.mil/sti/pdfs/ADA039677.pdf
- Zhang, T. (2005). Influence of the seasonal snow cover on the ground thermal regime: An overview. Reviews of Geophysics, 43(4). https://doi.org/https://doi.org/10.1029/2004RG000157![image-2.png](attachment:image-2.png)
